**Connect google drive**

In [ ]:
# To Check GPU
!nvidia-smi

Mon Jul 10 10:38:18 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# For mounting Drive to Google Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

**1) Clone the Darknet**



In [ ]:
# Clone the Darknet to the Colab
!git clone https://github.com/AlexeyAB/darknet

fatal: destination path 'darknet' already exists and is not an empty directory.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

[Errno 20] Not a directory: 'darknet'
/content/darknet
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
  946 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
 1147 |           

**3) Configure Darknet network for training YOLO V3**

In [ ]:
# Configuration file (cfg file)
!cp cfg/yolov3-tiny.cfg cfg/yolov3_tiny_box.cfg

In [ ]:
# Setting for single object

!sed -i 's/batch=1/batch=8/' cfg/yolov3_tiny_box.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_tiny_box.cfg
!sed -i 's/max_batches = 500200/max_batches = 3000/' cfg/yolov3_tiny_box.cfg
!sed -i '136 s@classes=80@classes=1@' cfg/yolov3_tiny_box.cfg
!sed -i '179 s@classes=80@classes=1@' cfg/yolov3_tiny_box.cfg
!sed -i '128 s@filters=255@filters=18@' cfg/yolov3_tiny_box.cfg
!sed -i '173 s@filters=255@filters=18@' cfg/yolov3_tiny_box.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
# !mkdir "/mydrive/yolov3_tiny/"

In [ ]:
# Jet is a class name
!echo "box" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3_tiny/' > data/obj.data
!mkdir data/obj

mkdir: cannot create directory ‘data/obj’: File exists


In [ ]:
!wget https://pjreddie.com/media/files/yolov3-tiny.weights

--2023-07-10 10:58:54--  https://pjreddie.com/media/files/yolov3-tiny.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35434956 (34M) [application/octet-stream]
Saving to: ‘yolov3-tiny.weights’

yolov3-tiny.weights 100%[===================>]  33.79M  12.2MB/s    in 2.8s    

2023-07-10 10:58:58 (12.2 MB/s) - ‘yolov3-tiny.weights’ saved [35434956/35434956]



In [ ]:
# Download weights darknet model for yolov3 tiny
!./darknet partial cfg/yolov3-tiny.cfg yolov3-tiny.weights yolov3-tiny.conv.15 15

 CUDA-version: 11080 (12000), cuDNN: 8.9.0, GPU count: 1  
 OpenCV version: 4.2.0
 GPU isn't used 
net.optimized_memory = 0 
mini_batch = 1, batch = 1, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     16       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  16 0.150 BF
   1 Create CUDA-stream - 0 
max                2x 2/ 2    416 x 416 x  16 ->  208 x 208 x  16 0.003 BF
   2 conv     32       3 x 3/ 1    208 x 208 x  16 ->  208 x 208 x  32 0.399 BF
   3 max                2x 2/ 2    208 x 208 x  32 ->  104 x 104 x  32 0.001 BF
   4 conv     64       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  64 0.399 BF
   5 max                2x 2/ 2    104 x 104 x  64 ->   52 x  52 x  64 0.001 BF
   6 conv    128       3 x 3/ 1     52 x  52 x  64 ->   52 x  52 x 128 0.399 BF
   7 max                2x 2/ 2     52 x  52 x 128 ->   26 x  26 x 128 0.000 BF
   8 conv    256       3 x 3/ 1     26 x  26 x 128 ->   26 x  26 x 256 0.399 BF
   9 max 

**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
# Extracting custom object dataset
!unzip /mydrive/yolov3_tiny/images.zip -d data/obj

Archive:  /mydrive/yolov3_tiny/images.zip
replace data/obj/images/default_gzclient_camera-1-2023-06-16T12_08_00-560873_jpg.rf.69e906126bc1f7013e5cbf5db5080b7b.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: data/obj/images/default_gzclient_camera-1-2023-06-16T12_08_00-560873_jpg.rf.69e906126bc1f7013e5cbf5db5080b7b.jpg  
  inflating: data/obj/images/default_gzclient_camera-1-2023-06-16T12_08_00-560873_jpg.rf.69e906126bc1f7013e5cbf5db5080b7b.txt  
  inflating: data/obj/images/default_gzclient_camera-1-2023-06-16T12_17_08-881012_jpg.rf.d1749de93389c193106ed0add239941c.jpg  
  inflating: data/obj/images/default_gzclient_camera-1-2023-06-16T12_17_08-881012_jpg.rf.d1749de93389c193106ed0add239941c.txt  
  inflating: data/obj/images/default_gzclient_camera-1-2023-06-16T12_31_54-865436_jpg.rf.6c5e3651fd062d2c1ee1aebafd86c33c.jpg  
  inflating: data/obj/images/default_gzclient_camera-1-2023-06-16T12_31_54-865436_jpg.rf.6c5e3651fd062d2c1ee1aebafd86c33c.txt  
  inflating: data/obj/images

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

In [ ]:
import glob
images_list = glob.glob("data/obj/images/*.jpg")
print(images_list)

['data/obj/images/default_gzclient_camera-1-2023-06-16T12_17_15-798663_jpg.rf.ddbe1c1d64fd3c0da55b3b4f582ba474.jpg', 'data/obj/images/default_gzclient_camera-1-2023-06-16T12_13_33-809409_jpg.rf.617e3f70639ca3647b5e9a5cdc09e679.jpg', 'data/obj/images/default_gzclient_camera-1-2023-06-16T12_36_11-904363_jpg.rf.80efb160bc5bc0aac64df4edca613590.jpg', 'data/obj/images/default_gzclient_camera-1-2023-06-16T12_19_47-730303_jpg.rf.6ceeeb1feeab7c7a1bf3980db874a86b.jpg', 'data/obj/images/default_gzclient_camera-1-2023-06-16T12_19_47-730303_jpg.rf.9142d0bb76173493db150dc0da7c66e1.jpg', 'data/obj/images/default_gzclient_camera-1-2023-06-16T12_13_49-301264_jpg.rf.ac7cee31ffac17d5dc626029baf9f324.jpg', 'data/obj/images/default_gzclient_camera-1-2023-06-16T12_36_11-904363_jpg.rf.df98a432b2a625425d1383fe2e686715.jpg', 'data/obj/images/default_gzclient_camera-1-2023-06-16T12_17_08-881012_jpg.rf.d1749de93389c193106ed0add239941c.jpg', 'data/obj/images/default_gzclient_camera-1-2023-06-16T12_36_21-920236_j

In [ ]:
#Create training.txt file, required for YOLOv3 training
file = open("data/train.txt", "w")
file.write("\n".join(images_list))
file.close()

In [ ]:
# Calculates anchors for the custom dataset
!./darknet detector calc_anchors data/obj.data -num_of_clusters 1 -width 640 -height 640

 CUDA-version: 11080 (12000), cuDNN: 8.9.0, GPU count: 1  
 OpenCV version: 4.2.0

 num_of_clusters = 1, width = 640, height = 640 
 read labels from 127 images 
 loaded 	 image: 127 	 box: 127
 all loaded. 

 calculating k-means++ ...

 iterations = 0 


counters_per_class = 127

 avg IoU = 50.91 % 

Saving anchors to the file: anchors.txt 
anchors = 159,250
^C


**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_tiny_box.cfg yolov3-tiny.conv.15 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 23 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 45792, rewritten_bbox = 0.000000 % 
]2;2903/3000: loss=0.1 hours left=0.0
 2903: 0.136614, 0.151628 avg loss, 0.001000 rate, 0.387131 seconds, 46448 images, 0.014590 hours left
Loaded: 0.000044 seconds
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 16 Avg (IOU: 0.844168), count: 1, class_loss = 0.145373, iou_loss = 0.116678, total_loss = 0.262051 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 23 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 45793, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 16 Avg (IOU: 0.732441), count: 1, class_loss = 0.000001, iou_loss = 0.106875, total_loss = 0.106875 
v3 (mse l

In [ ]:
imShow('chart.png')

NameError: ignored

In [ ]:
# from google.colab import files
# files.download("model.json")

In [ ]:
# from google.colab import files

# #For model json
# model_json = model.to_json()
# with open("model1.json","w") as json_file:
#      json_file.write(model_json)
# files.download("model1.json")

# #For weights
# model.save('weights.h5')
# files.download('weights.h5')

In [ ]:
# Training on last saved weights
# !./darknet detector train data/obj.data cfg/yolov3_training.cfg '/content/gdrive/MyDrive/yolov3/yolov3_training_last.weights' -dont_show

In [ ]:
# model.save_weights('my_model_weights.h5')
# model.save('my_model.h5')

In [ ]:
!pwd

In [ ]:
# Testing on an image
# !./darknet detect cfg/yolov3_testing.cfg '/content/gdrive/MyDrive/yolov3/yolov3_training_final.weights' data/Test_img.jpg
# !./darknet detect cfg/yolov3_training.cfg '/content/gdrive/MyDrive/yolov3/yolov3_training_final.weights' data/Test_img.jpg

In [ ]:
# Mean Average Precision (mAP) Calculation
# !./darknet detector map data/obj.data cfg/yolov3_training.cfg '/content/gdrive/mydrive/yolov3/yolov3_trained.weights'